In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [3]:
def soft_grouping(S, n):
    m = len(S)

    # Create a Gurobi model
    model = gp.Model()
    model.setParam("NonConvex", 2)
    model.setParam("OutputFlag", 0)

    # Create binary decision variables
    X = model.addVars(m, n, vtype=GRB.BINARY, name="X")

    # Create continuous decision variables for y
    y = model.addVars(n, vtype=GRB.CONTINUOUS, name="y")

    z = model.addVars(m, n, m, vtype=GRB.BINARY, name="z")

    objective_expr = gp.quicksum(
        y[j]
        * gp.quicksum(
            [gp.quicksum([z[i, j, k] * S[i, k] for i in range(m)]) for k in range(m)]
        )
        for j in range(n)
    )

    model.setObjective(objective_expr, sense=GRB.MAXIMIZE)

    # Each task is assigned to exactly one group
    for i in range(m):
        model.addConstr(
            gp.quicksum(X[i, j] for j in range(n)) >= 1, name=f"constraint1_{i}"
        )

    # introduce the auxiliary variable y
    for j in range(n):
        model.addConstr(
            gp.quicksum(X[i, j] for i in range(m)) * y[j] == 1,
            name=f"constraint2_{j}",
        )

    # each group has at least one task
    for j in range(n):
        model.addConstr(
            gp.quicksum(X[i, j] for i in range(m)) >= 1, name=f"constraint3_{j}"
        )

    # no same groups
    for j1 in range(n):
        for j2 in range(j1, n):
            if j1 != j2:
                model.addConstr(
                    gp.quicksum(
                        X[i, j1] + X[i, j2] - 2 * X[i, j1] * X[i, j2] for i in range(m)
                    )
                    >= 1,
                    name=f"constraint4_{j1}_{j2}",
                )

    # introduce z to relax to a quadratic program
    for i in range(m):
        for j in range(n):
            for k in range(m):
                model.addConstr(
                    z[i, j, k] - X[i, j] * X[k, j] == 0, name=f"constraint5_{i}_{j}_{k}"
                )

    # Optimize the model
    model.optimize()
    print('Sovling Time: {:.3f}s'.format(model.Runtime))

    groups = [[] for i in range(n)]
    soln = np.zeros((m, n))
    # Print the solution
    if model.status == GRB.OPTIMAL:
        for j in range(n):
            for i in range(m):
                soln[i, j] = X[i, j].X
                if np.isclose(X[i, j].X, 1):
                    groups[j].append(i)
    else:
        print("No optimal solution found.")
    return groups, model.objVal


